In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import altair as alt

In [7]:
data = pd.read_csv('Data/shap_values_with_coordinates.csv')

In [10]:
data['index'] = data.index

In [17]:
data.to_csv('Data/shap_values_with_coordinates_indexed.csv')

In [16]:
brush = alt.selection_interval()

points = alt.Chart(data).mark_circle().encode(
    x='x:Q',
    y='y:Q',
    color=alt.Color('class:O').scale(scheme='set1'),
    opacity=alt.condition(brush, alt.value(1), alt.value(0.2))
).add_params(brush).properties(width=750, height=750)

ranked_text = alt.Chart(data).mark_text(align='right').encode(
    y=alt.Y('row_number:O').axis(None)
).transform_filter(
    brush
).transform_window(
    row_number='row_number()'
).transform_filter(
    alt.datum.row_number < 150
)

features = ['index'] + [x for x in data.columns if x not in ['x','y', 'index']]
columns = []

for feature in features:
    feature_var = ranked_text.encode(text=alt.Text(f'{feature}:Q')).properties(
        title=alt.Title(text=feature.capitalize(), align='right'),
    )
    columns.append(feature_var)

alt.vconcat(points, alt.hconcat(*columns)).save('test.html')